In [2]:
! wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz

--2020-07-05 10:27:16--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   3.41G  8.97MB/s    in 6m 23s  

2020-07-05 10:33:42 (9.11 MB/s) - ‘en.txt.gz’ saved [3663376519/3663376519]



In [3]:
!gunzip -k en.txt.gz
!mkdir lines
!split -a 3 -l 100000 en.txt lines/lines-

zip

In [2]:
import re

def _should_skip(line, min_length, max_length):
    """Whether a line should be skipped depending on the length."""
    return len(line) < min_length or len(line) > max_length


def create_example(previous_lines, line, file_id):
    """Creates examples with multi-line context
    The examples will include:
        file_id: the name of the file where these lines were obtained.
        response: the current line text
        context: the previous line text
        context/0: 2 lines before
        context/1: 3 lines before, etc.
    """
    example = {
        'file_id': file_id,
        'context': previous_lines[-1],
        'response': line,
    }
    example['file_id'] = file_id
    example['context'] = previous_lines[-1]

    extra_contexts = previous_lines[:-1]
    example.update({
        'context/{}'.format(i): context
        for i, context in enumerate(extra_contexts[::-1])
    })

    return example


def _preprocess_line(line):
    line = line.decode("utf-8")

    # Remove the first word if it is followed by colon (speaker names)
    # NOTE: this wont work if the speaker's name has more than one word
    line = re.sub('(?:^|(?:[.!?]\\s))(\\w+):', "", line)

    # Remove anything between brackets (corresponds to acoustic events).
    line = re.sub("[\\[(](.*?)[\\])]", "", line)

    # Strip blanks hyphens and line breaks
    line = line.strip(" -\n")

    return line


def _create_examples_from_file(file_name, min_length=0, max_length=20,
                               num_extra_contexts=5):

    previous_lines = []
    with open(file_name, 'rb') as f:
      for line in f :
        line = _preprocess_line(line)
        if not line:
            continue

        should_skip = _should_skip(
            line,
            min_length=min_length,
            max_length=max_length)

        if previous_lines:
            should_skip |= _should_skip(
                previous_lines[-1],
                min_length=min_length,
                max_length=max_length)

            if not should_skip:
                yield create_example(previous_lines, line, file_name)

        previous_lines.append(line.lower())
        if len(previous_lines) > num_extra_contexts + 1:
            del previous_lines[0]

In [3]:
example = _create_examples_from_file(file_name='lines/lines-aaa')

count = 0
for i in example:
  count += 1
print('Found '+ str(count*5) + ' examples')
print(i)

Found 101770 examples
{'file_id': 'lines/lines-aaa', 'context': "we'll catch him.", 'response': 'Because we have to.', 'context/0': 'oh, aye.', 'context/1': "tell me, sergeant, in your professional opinion, do you really believe you can catch this man if he doesn't want you to?", 'context/2': 'i have.', 'context/3': 'you seem to have made a great study of crime, mr. newspaperman.', 'context/4': "the ones we don't know about yet."}


In [4]:
in_comma = "'"

def remove_char(sentence):
  sent = sentence.replace('!', '')
  sent = sent.replace(',', '')
  sent = sent.replace(in_comma, '')
  sent = sent.replace('%', '')
  sent = sent.replace('-', '')
  sent = sent.replace('.', '')
  sent = sent.replace('?', '')
  sent = sent.replace('/', '')
  sent = sent.replace(':', '')
  sent = sent.replace(';', '')

  return sent

ex_sent = 'Hello, me! why?'
print(remove_char(ex_sent).lower())

hello me why


In [5]:

example = _create_examples_from_file(file_name='lines/lines-aaa')

inputs = []
responses = []
i = 0

for test in example:

  input_1 = test['context']
  input_2 = test['context/0']
  input_3 = test['context/1']
  input_4 = test['context/2']
  input_5 = test['context/3']
  response = test['response']
    
  inputs.append(remove_char(input_1))
  inputs.append(remove_char(input_2))
  inputs.append(remove_char(input_3))
  inputs.append(remove_char(input_4))
  inputs.append(remove_char(input_5))

  for j in range(5):
    responses.append('startsent' + ' ' +remove_char(response.lower())+ ' ' + 'endsent')
    
  i += 5

print(inputs[101769], responses[101769])

you seem to have made a great study of crime mr newspaperman startsent because we have to endsent


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

oov_token = "<OOV>"
max_length = 20
num_topic_words = 4

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(inputs)
tokenizer.fit_on_texts(responses)

word_index = tokenizer.word_index
word_index['startsent'] = 0
word_index['endsent'] = len(word_index)+1
index_word = {word_index[word]:word for word in word_index}
#print(index_word)
vocab_size = len(word_index) + 1
input_seq = tokenizer.texts_to_sequences(inputs)
response_seq = tokenizer.texts_to_sequences(responses)
input_seq_pad = pad_sequences(input_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
response_seq_pad = pad_sequences(response_seq, maxlen = max_length*2, padding = 'post', truncating = 'post')

In [7]:
def preprocess_sent(text_list):
  inputs = []
  for sent in text_list:
    inputs.append(remove_char(sent))
  input_seq = tokenizer.texts_to_sequences(inputs)
  input_seq_pad = pad_sequences(input_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
  return input_seq_pad  

In [10]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip


--2020-07-05 10:45:20--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-07-05 10:45:20--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-07-05 10:45:21--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [154]:
from zipfile import ZipFile 
with ZipFile('glove.42B.300d.zip', 'r') as mad: 
  mad.extractall() 


<zipfile.ZipFile [closed]>

<zipfile.ZipFile [closed]>

In [8]:
import os
import numpy as np

embeddings_index = {} 
f = open('/content/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [9]:
embedding_dim = 300

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))            
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

print(len(embeddings_matrix))
embeddings_matrix[6977]

6980


array([ 3.36530000e-01, -3.35099995e-01,  6.26939982e-02, -2.94600010e-01,
       -2.73090005e-01,  9.11000013e-01, -2.82599998e+00,  5.84870018e-02,
       -4.92949992e-01,  1.42220005e-01,  3.02810013e-01,  7.64840007e-01,
       -3.16020012e-01, -5.04109979e-01, -2.34589994e-01,  2.64039993e-01,
       -1.02569997e-01,  3.87710005e-01, -2.36750007e-01,  1.19010001e-01,
        9.07519996e-01,  4.25880015e-01, -1.00560002e-01,  7.10859969e-02,
       -6.39040023e-02,  8.33090007e-01, -1.13660000e-01, -5.71449995e-01,
       -2.29409993e-01,  4.60310012e-01, -4.93079990e-01, -7.80960023e-02,
        7.69479990e-01, -5.64499974e-01,  1.93869993e-01,  4.83650006e-02,
       -1.18450001e-01, -1.75060004e-01,  3.09949994e-01,  7.11000025e-01,
        1.14929996e-01, -4.95620012e-01, -2.78039992e-01, -4.06749994e-01,
       -4.02310014e-01,  4.94029999e-01,  1.76760003e-01,  3.64100009e-01,
       -2.72269994e-01,  1.20540000e-01, -2.18170002e-01,  2.84779996e-01,
       -4.83669996e-01,  

In [10]:
#Implementing Lda topic modeling
#Splitting the inputs list itens word wise
topic_train = []
for i in range(len(inputs)):
  topic_train.append(inputs[i].split())

import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')


#Filter out stop words in English 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
for i in range(len(topic_train)):
  sent=" ".join(topic_train[i])
  sent=nlp(sent)
  topic_train[i] = [ stemmer.stem(w.lemma_) for w in sent if w.text.lower() not in stops and len(w)>2]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#Forming dictionary out of the words
from gensim import corpora
topic_dict = corpora.Dictionary(topic_train)
topic_dict.filter_extremes(no_below=10, no_above=0.5)

#Obtaining bow_corpus out of the topics
bow_corpus = [topic_dict.doc2bow(doc) for doc in topic_train]

#Getting tfidf corpus from tfidf model
from gensim import models
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

#training the lda model
from gensim import models
NUM_TOPICS = 30
NUM_PASSES = 60 
ALPHA = 'auto'
ETA = 'auto'
# Train LDA model
lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus, 
                                         num_topics=NUM_TOPICS, 
                                         id2word=topic_dict,
                                         passes=NUM_PASSES, 
                                         alpha=ALPHA, 
                                         eta=ETA,
                                         random_state=200)


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [12]:
print(inputs[5],responses[4])
print(inputs[5:10],responses[1])

here we go startsent here we go endsent
['here we go', 'im going to wait', 'we promised to meet here two years later but she hasnt come yet', 'exactly two years ago today she and i buried a time capsule here', 'my sassy girl'] startsent here we go endsent


In [13]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

print(topic_train[0])

#get the scores of the topics generated
for index, score in sorted(lda_model[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

[(19,
  '0.119*"hand" + 0.099*"minut" + 0.051*"jump" + 0.048*"fast" + 0.044*"murder" '
  '+ 0.040*"peopl" + 0.035*"kiss" + 0.033*"save" + 0.030*"air" + '
  '0.029*"realiz"'),
 (14,
  '0.139*"understand" + 0.125*"wait" + 0.113*"die" + 0.091*"woman" + '
  '0.072*"saito" + 0.054*"first" + 0.040*"war" + 0.032*"punish" + '
  '0.023*"prepar" + 0.021*"pretti"'),
 (22,
  '0.200*"bridg" + 0.136*"pleas" + 0.100*"move" + 0.042*"life" + 0.042*"build" '
  '+ 0.042*"possibl" + 0.038*"spend" + 0.036*"veronica" + 0.032*"someon" + '
  '0.030*"husband"'),
 (6,
  '0.244*"tell" + 0.085*"tri" + 0.081*"finish" + 0.079*"readi" + 0.075*"never" '
  '+ 0.038*"studi" + 0.035*"chapman" + 0.031*"captain" + 0.031*"kid" + '
  '0.027*"fault"'),
 (13,
  '0.154*"back" + 0.111*"talk" + 0.065*"bori" + 0.063*"idea" + 0.054*"end" + '
  '0.054*"beauti" + 0.049*"job" + 0.047*"goodby" + 0.036*"ever" + '
  '0.035*"interest"'),
 (24,
  '0.303*"see" + 0.181*"help" + 0.157*"sorri" + 0.049*"babi" + 0.029*"intrud" '
  '+ 0.028*"fal

In [60]:
'''#Implementing Lda topic modeling
#Splitting the inputs list itens word wise
topic_train = []
for i in range(len(inputs)):
  topic_train.append(inputs[i].split())

import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')


#Filter out stop words in English 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
for i in range(len(topic_train)):
  sent=" ".join(topic_train[i])
  sent=nlp(sent)
  topic_train[i] = [ w.lemma_ for w in sent if w.text.lower() not in stops and len(w)>2]

'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
#Forming dictionary out of the words
from gensim import corpora
topic_dict = corpora.Dictionary(topic_train)
topic_dict.filter_extremes(no_below=10, no_above=0.5)

#Obtaining bow_corpus out of the topics
bow_corpus = [topic_dict.doc2bow(doc) for doc in topic_train]

#Getting tfidf corpus from tfidf model
from gensim import models
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

#training the lda model
from gensim import models
NUM_TOPICS = 30
NUM_PASSES = 60 
ALPHA = 'auto'
ETA = 'auto'
# Train LDA model
lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus, 
                                         num_topics=NUM_TOPICS, 
                                         id2word=topic_dict,
                                         passes=NUM_PASSES, 
                                         alpha=ALPHA, 
                                         eta=ETA,
                                         random_state=200)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [62]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

print(topic_train[0])

#get the scores of the topics generated
for index, score in sorted(lda_model[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

[(1,
  '0.229*"help" + 0.150*"understand" + 0.101*"hear" + 0.099*"woman" + '
  '0.080*"darle" + 0.042*"people" + 0.035*"head" + 0.030*"realize" + '
  '0.020*"piece" + 0.017*"maam"'),
 (25,
  '0.131*"name" + 0.105*"warden" + 0.076*"wonder" + 0.063*"list" + '
  '0.042*"report" + 0.039*"shut" + 0.035*"group" + 0.034*"principle" + '
  '0.033*"fault" + 0.026*"wilderness"'),
 (3,
  '0.130*"colonel" + 0.075*"mama" + 0.073*"hope" + 0.071*"lovely" + '
  '0.069*"boy" + 0.069*"believe" + 0.053*"jolly" + 0.045*"stella" + '
  '0.044*"shall" + 0.027*"tired"'),
 (8,
  '0.167*"back" + 0.120*"talk" + 0.084*"feel" + 0.070*"forget" + '
  '0.064*"suppose" + 0.060*"catch" + 0.032*"meg" + 0.030*"sort" + '
  '0.024*"fever" + 0.023*"dysentery"'),
 (11,
  '0.067*"soldier" + 0.057*"gold" + 0.049*"kind" + 0.047*"hit" + '
  '0.046*"beautiful" + 0.045*"quite" + 0.043*"together" + 0.043*"first" + '
  '0.040*"car" + 0.039*"horse"'),
 (5,
  '0.168*"show" + 0.083*"maybe" + 0.073*"order" + 0.059*"many" + 0.034*"light" 

In [78]:
'''#Implementing Lda topic modeling
#Splitting the inputs list itens word wise
topic_train = []
for i in range(len(inputs)):
  topic_train.append(inputs[i].split())

import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')


#Filter out stop words in English 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
for i in range(len(topic_train)):
  sent=" ".join(topic_train[i])
  sent=nlp(sent)
  topic_train[i] = [ w.lemma_ for w in sent if w.text.lower() not in stops and len(w)>2]
'''


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [79]:
#Forming dictionary out of the words
from gensim import corpora
topic_dict = corpora.Dictionary(topic_train)
topic_dict.filter_extremes(no_below=10, no_above=0.5)

#Obtaining bow_corpus out of the topics
bow_corpus = [topic_dict.doc2bow(doc) for doc in topic_train]

#Getting tfidf corpus from tfidf model
from gensim import models
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

#training the lda model
from gensim import models
NUM_TOPICS = 30
NUM_PASSES = 60 
ALPHA = 'auto'
ETA = 'auto'
# Train LDA model
lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus, 
                                         num_topics=NUM_TOPICS, 
                                         id2word=topic_dict,
                                         passes=NUM_PASSES, 
                                         alpha=ALPHA, 
                                         eta=ETA,
                                         random_state=200)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [80]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

print(topic_train[0])

#get the scores of the topics generated
for index, score in sorted(lda_model[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

[(1,
  '0.229*"help" + 0.150*"understand" + 0.101*"hear" + 0.099*"woman" + '
  '0.080*"darle" + 0.042*"people" + 0.035*"head" + 0.030*"realize" + '
  '0.020*"piece" + 0.017*"maam"'),
 (25,
  '0.131*"name" + 0.105*"warden" + 0.076*"wonder" + 0.063*"list" + '
  '0.042*"report" + 0.039*"shut" + 0.035*"group" + 0.034*"principle" + '
  '0.033*"fault" + 0.026*"wilderness"'),
 (3,
  '0.130*"colonel" + 0.075*"mama" + 0.073*"hope" + 0.071*"lovely" + '
  '0.069*"boy" + 0.069*"believe" + 0.053*"jolly" + 0.045*"stella" + '
  '0.044*"shall" + 0.027*"tired"'),
 (8,
  '0.167*"back" + 0.120*"talk" + 0.084*"feel" + 0.070*"forget" + '
  '0.064*"suppose" + 0.060*"catch" + 0.032*"meg" + 0.030*"sort" + '
  '0.024*"fever" + 0.023*"dysentery"'),
 (11,
  '0.067*"soldier" + 0.057*"gold" + 0.049*"kind" + 0.047*"hit" + '
  '0.046*"beautiful" + 0.045*"quite" + 0.043*"together" + 0.043*"first" + '
  '0.040*"car" + 0.039*"horse"'),
 (5,
  '0.168*"show" + 0.083*"maybe" + 0.073*"order" + 0.059*"many" + 0.034*"light" 

In [14]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.porter import *
import numpy as np
nltk.download('wordnet')

stemmer = PorterStemmer()

def lemmatize_stemming(text):
    x = WordNetLemmatizer().lemmatize(text, pos='v')
    return stemmer.stem(x)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import random
def extract_topic(input):

  c = 0
  bow_vector = topic_dict.doc2bow(preprocess(input))
  for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
      wp = lda_model.show_topic(index)
      topic_keywords = ", ".join([word for word, prop in wp])
      '''if c == random.randint(0,30):
         break
      c += 1'''
  return topic_keywords

#input = 'We are really blessed by God with a caring and loving mother. Without mothers our lives are nothing. We are so lucky as we have a mother. We give lots of special gifts to our mother and she gives us lots of love and care.'
extract_topic('i will kill him')

'hand, minut, jump, fast, murder, peopl, kiss, save, air, realiz'

In [16]:
good=extract_topic('i want to kill him').split(', ')
print(input_seq_pad[0], response_seq_pad[0])
print(extract_topic(inputs[0]),"----",extract_topic(responses[0]))

[ 28  79   7 106   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0] [   0   21   40   20 6979    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
hand, minut, jump, fast, murder, peopl, kiss, save, air, realiz ---- hand, minut, jump, fast, murder, peopl, kiss, save, air, realiz


In [17]:
###
new_input_seq_pad = []
new_response_seq_pad = []

'''for i in range(len(inputs)):
    topics = extract_topic(inputs[i])
    topics = topics.split(', ')
    topic_seq = tokenizer.texts_to_sequences([topics[:num_topic_words]])
    topic_seq_pad = pad_sequences(topic_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
    temp = np.concatenate((input_seq_pad[i], np.array(topic_seq_pad[0])))
    new_input_seq_pad.append(temp) '''
for i in range(len(inputs)):
    topics = extract_topic(inputs[i])
    topics_responses=extract_topic(responses[i])
    topics = topics.split(', ')
    topics_responses=topics_responses.split(', ')
    topic_seq = tokenizer.texts_to_sequences([topics[:num_topic_words]])
    topic_seq_responses = tokenizer.texts_to_sequences([topics_responses[:num_topic_words]])
    topic_seq_pad = pad_sequences(topic_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
    topic_seq_pad_responses = pad_sequences(topic_seq_responses, maxlen = max_length ,padding = 'post', truncating = 'post')
    temp = np.concatenate((input_seq_pad[i], np.array(topic_seq_pad[0])))
    temp_final=np.concatenate((response_seq_pad[i], np.array(topic_seq_pad_responses[0])))
    new_input_seq_pad.append(temp)
    new_response_seq_pad.append(temp_final) 

    
print(len(word_index))
print(inputs[0], responses[0])
print(new_input_seq_pad[0], new_response_seq_pad[0])
print(len(new_input_seq_pad))
print(new_input_seq_pad[1], new_response_seq_pad[1])


6978
im going to wait startsent here we go endsent
[ 28  79   7 106   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0 316   1 495 457   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0] [   0   21   40   20 6979    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  316    1
  495  457    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
101770
[  40  784    7  470   21  138  192  281   60  119 1063   26  280    0
    0    0    0    0    0    0  316    1  495  457    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0] [   0   21   40   20 6979    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  316    1
  495  457    0    0    0    0    0    0    0    0    0

In [18]:
print(topic_seq_pad_responses)
print(new_input_seq_pad[1], new_response_seq_pad[0])

[[316   1 495 457   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
[  40  784    7  470   21  138  192  281   60  119 1063   26  280    0
    0    0    0    0    0    0  316    1  495  457    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0] [   0   21   40   20 6979    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  316    1
  495  457    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [19]:
def get_topic(input):

    input_topic = extract_topic(input)
    input_seq = tokenizer.texts_to_sequences([input_topic])
    input_seq_pad = pad_sequences(input_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
    return input_seq_pad[0]

get_topic('I am happy')

array([316,   1, 495, 457, 652,   1, 437, 540, 423,   1,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [20]:

"""print(input_seq_pad.shape)
X_train = input_seq_pad[:500]
y_train = response_seq_pad[:500]"""

X_train = []
y_train = []
for j in range(50000):
    X_train.append(new_input_seq_pad[j])
    y_train.append(new_response_seq_pad[j])

X_train = np.array(X_train)
y_train = np.array(y_train)

print(embedding_dim)
print(X_train.shape)
print(y_train.shape)

dataset = (inputs, responses)
print(len(inputs))

300
(50000, 40)
(50000, 60)
101770


In [21]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GRU, Dense 

max_sequence_len = 40
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

train_dataset

<PrefetchDataset shapes: ((64, 40), (64, 60)), types: (tf.int32, tf.int32)>

In [58]:
class Encoder(tf.keras.Model):
    def __init__(self, hidden_size=1024, max_sequence_len=40, batch_size=batch_size, embedding_dim=300, vocab_size=vocab_size+1):
        super(Encoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.max_sequence_len = max_sequence_len
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embedding_layer = Embedding(
            input_dim=self.vocab_size, output_dim=self.embedding_dim, weights=[embeddings_matrix], trainable=False)
        self.GRU_1 = GRU(units=hidden_size, return_sequences=True,recurrent_initializer='glorot_uniform')
        self.GRU_2 = GRU(units=hidden_size,
                         return_sequences=True, return_state=True,recurrent_initializer='glorot_uniform')

    def initial_hidden_state(self):
        return tf.zeros(shape=(self.batch_size, self.hidden_size))

    def call(self, x, initial_state, training=False):
        x = self.embedding_layer(x)
        x = self.GRU_1(x, initial_state=initial_state)
        x, hidden_state = self.GRU_2(x)
        return x, hidden_state

In [59]:

class Attention(tf.keras.Model):
    def __init__(self, hidden_size=256):
        super(Attention, self).__init__()
        self.fc1 = Dense(units=hidden_size)
        self.fc2 = Dense(units=hidden_size)
        self.fc3 = Dense(units=1)

    def call(self, encoder_output, hidden_state, training=False):
        '''hidden_state : h(t-1)'''
        y_hidden_state = tf.expand_dims(hidden_state, axis=1)
        y_hidden_state = self.fc1(y_hidden_state)
        y_enc_out = self.fc2(encoder_output)

        y = tf.keras.backend.tanh(y_enc_out + y_hidden_state)
        attention_score = self.fc3(y)
        attention_weights = tf.keras.backend.softmax(attention_score, axis=1)

        context_vector = tf.multiply(encoder_output, attention_weights)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [60]:
class Decoder(tf.keras.Model):
    def __init__(self, hidden_size=1024, max_sequence_len=40, batch_size=batch_size, embedding_dim=300, vocab_size=vocab_size+1):
        super(Decoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.max_sequence_len = max_sequence_len
        self.hidden_size = hidden_size
        self.batch_size = batch_size
    
        self.embedding_layer = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, weights=[embeddings_matrix], trainable=False)
        self.GRU = GRU(units=hidden_size,
                       return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.attention = Attention(hidden_size=self.hidden_size)
        self.fc = Dense(units=self.vocab_size)

    def initial_hidden_state(self):
        return tf.zeros(shape=(self.batch_size, self.hidden_size))

    def call(self, x, encoder_output, hidden_state, training=False):
        x = self.embedding_layer(x)
        context_vector, attention_weights = self.attention(
            encoder_output, hidden_state, training=training)
        contect_vector = tf.expand_dims(context_vector, axis=1)
        x = tf.concat([x, contect_vector], axis=-1)
        x, curr_hidden_state = self.GRU(x)
        x = tf.reshape(x, shape=[self.batch_size, -1])
        x = self.fc(x)
        return x, curr_hidden_state, attention_weights

In [61]:
loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
train_accuracy = tf.metrics.SparseCategoricalAccuracy()

def loss_function(y_true, y_pred):
    loss = loss_object(y_true, y_pred)
    mask = 1 - tf.cast(tf.equal(y_true, 0), 'float32')
    return tf.reduce_mean(loss * mask)

In [62]:
def training_step(inputs, responses):    
    with tf.GradientTape() as Tape:
        encoder_init_state = encoder.initial_hidden_state()
        encoder_output, encoder_hidden_state = encoder(inputs, encoder_init_state, training=True)
        decoder_hidden = encoder_hidden_state
        loss = 0
        acc = []
        current_word = tf.expand_dims(responses[:, 0], axis=1)
        for word_idx in range(1, max_sequence_len):
            next_word = responses[:, word_idx]
            logits, decoder_hidden, attention_weights = decoder(current_word, encoder_output, decoder_hidden)
            loss += loss_function(next_word, logits)
            acc.append(train_accuracy(next_word, logits))
            current_word = tf.expand_dims(next_word, axis=1)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = Tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss, tf.reduce_mean(acc)

encoder = Encoder()
decoder = Decoder()

In [65]:
epochs = 15
NUM_SAMPLES = len(X_train)
num_steps = NUM_SAMPLES // batch_size

for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    ep_loss = []
    ep_acc = []
    progbar = tf.keras.utils.Progbar(target=num_steps, stateful_metrics=[
                                     'curr_loss', 'curr_accuracy'], unit_name='batch')

    for step, example in enumerate(train_dataset):
        inputs = example[0]
        responses = example[1]
        loss, acc = training_step(inputs, responses)
        loss /= responses.shape[1]
        ep_loss.append(loss)
        ep_acc.append(acc)
        progbar.update(
            step + 1, values=[('curr_loss', loss), ('curr_accuracy', acc)])

    print(f'Metrics after epoch {epoch} : Loss => {np.mean(ep_loss):.3f} | Accuracy => {np.mean(ep_acc):.3f}')

Epoch 1/15
781/781 [==============================] - 537s 687ms/batch - curr_loss: 0.1453 - curr_accuracy: 0.9224
Metrics after epoch 1 : Loss => 0.185 | Accuracy => 0.852
Epoch 2/15
781/781 [==============================] - 537s 688ms/batch - curr_loss: 0.1213 - curr_accuracy: 0.9377
Metrics after epoch 2 : Loss => 0.134 | Accuracy => 0.932
Epoch 3/15
781/781 [==============================] - 536s 686ms/batch - curr_loss: 0.1087 - curr_accuracy: 0.9443
Metrics after epoch 3 : Loss => 0.119 | Accuracy => 0.941
Epoch 4/15
781/781 [==============================] - 534s 684ms/batch - curr_loss: 0.1051 - curr_accuracy: 0.9485
Metrics after epoch 4 : Loss => 0.109 | Accuracy => 0.946
Epoch 5/15
781/781 [==============================] - 535s 685ms/batch - curr_loss: 0.1005 - curr_accuracy: 0.9516
Metrics after epoch 5 : Loss => 0.101 | Accuracy => 0.950
Epoch 6/15
781/781 [==============================] - 536s 686ms/batch - curr_loss: 0.0950 - curr_accuracy: 0.9542
Metrics after epoch 

In [29]:

def translate_sentence(sentence):
    topic_sent = get_topic(sentence)
    sentence = preprocess_sent([sentence])
    new_sentence = np.concatenate((np.array(sentence[0]), np.array(topic_sent)))
    enc_init = tf.zeros(shape=[1, 1024])
    enc_out, enc_hidden = encoder(sentence, enc_init)
    decoder.batch_size = 1
    tokenizer.index_word[0] = ''
    decoded = []
    att = []
    current_word = tf.expand_dims([word_index['startsent']], axis=0) 
    decoder_hidden = enc_hidden
    for word_idx in range(1, max_sequence_len):
        logits, decoder_hidden, attention_weights = decoder(current_word, enc_out, decoder_hidden)
        decoded_idx = np.argmax(logits)
        if index_word[decoded_idx] == 'endsent':
            break
        decoded.append(tokenizer.index_word[decoded_idx])
        att.append(attention_weights.numpy().squeeze())
        current_word = tf.expand_dims([decoded_idx], axis=0)
    return ' '.join(decoded), att

In [70]:

sentences = ['who are you','what do you do','where are you going','who do you live with','are you single','i know a good cafe here','you are very rude','is modi god','i want to be free']

for inp_sentence in sentences:
    inp_array = inp_sentence.split()
    inp_len = len(inp_sentence.split())
    trans_sentence, attention_weights = translate_sentence(inp_sentence)
    trans_array = trans_sentence.split()
    trans_len = len(trans_array)
    print('INPUT : ', inp_sentence)
    print('RESPONSE : ', trans_sentence)
    print('-'*30)

INPUT :  who are you
RESPONSE :  how old are you
------------------------------
INPUT :  what do you do
RESPONSE :  nurse
------------------------------
INPUT :  where are you going
RESPONSE :  im on duty
------------------------------
INPUT :  who do you live with
RESPONSE :  how old are you
------------------------------
INPUT :  are you single
RESPONSE :  what
------------------------------
INPUT :  i know a good cafe here
RESPONSE :  i cant
------------------------------
INPUT :  you are very rude
RESPONSE :  whats it good for
------------------------------
INPUT :  is modi god
RESPONSE :  hes going to live
------------------------------
INPUT :  i want to be free
RESPONSE :  call the doctor
------------------------------
